In [100]:
# Dependencies
import requests
from config import api_key, CLIENT_ID, CLIENT_SECRET
import json
import pprint as pp
import pandas as pd

In [101]:
#Playlist for reference: https://open.spotify.com/playlist/20VO4SBwxES4tXbp1nvjk6?si=2kxhbktBSPua1xQd0xbq3w
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [102]:
# Build query URL

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

url = "https://api.spotify.com/v1/"

play_id = "20VO4SBwxES4tXbp1nvjk6?si=2kxhbktBSPua1xQd0xbq3w"

query_url = url+'playlists/'+play_id+'tracks'

In [103]:
# Get playlist data
json_req = requests.get(query_url, headers=headers)
response = json_req.json()

track_ids = []
artist_ids = []
track_names = []
artist_names = []
genres =[]

In [107]:
#for item in response['tracks']['items']:
#    track_names.append(item['track']['name'])
#    track_ids.append(item['track']['id'])
#    album_names.append(item['track']['album']['name'])
#    album_ids.append(item['track']['album']['id'])
#    genres.append(item['track']['artists'][0]['genres'])
#df = pd.DataFrame({ 'Track Name':track_names, 'Album Name':album_names, 'Track ID':track_ids, 'Album ID': album_ids,  'Genres':genres})
#df.head()
pp.pprint(response['tracks']['items']['track'])

[{'added_at': '2023-05-25T02:00:21Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/clgvhwn4sxd81bxao64txfapn'},
               'href': 'https://api.spotify.com/v1/users/clgvhwn4sxd81bxao64txfapn',
               'id': 'clgvhwn4sxd81bxao64txfapn',
               'type': 'user',
               'uri': 'spotify:user:clgvhwn4sxd81bxao64txfapn'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_type': 'compilation',
                      'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of'},
                                   'href': 'https://api.spotify.com/v1/artists/0LyfQWJT6nXafLPZqxe9Of',
                                   'id': '0LyfQWJT6nXafLPZqxe9Of',
                                   'name': 'Various Artists',
                                   'type': 'artist',
                                   'uri': 'spotify:artist:0LyfQWJT6nXafLPZqxe9Of'}],
                      'available_mar

                         'type': 'artist',
                         'uri': 'spotify:artist:1HwM5zlC5qNWhJtM00yXzG'}],
            'available_markets': ['AR',
                                  'AU',
                                  'AT',
                                  'BE',
                                  'BO',
                                  'BR',
                                  'BG',
                                  'CA',
                                  'CL',
                                  'CO',
                                  'CR',
                                  'CY',
                                  'CZ',
                                  'DK',
                                  'DO',
                                  'DE',
                                  'EC',
                                  'EE',
                                  'SV',
                                  'FI',
                                  'FR',
                                  'GR',
  

                                  'SK',
                                  'ES',
                                  'SE',
                                  'CH',
                                  'TW',
                                  'TR',
                                  'UY',
                                  'US',
                                  'GB',
                                  'AD',
                                  'LI',
                                  'MC',
                                  'ID',
                                  'JP',
                                  'TH',
                                  'VN',
                                  'RO',
                                  'IL',
                                  'ZA',
                                  'SA',
                                  'AE',
                                  'BH',
                                  'QA',
                                  'OM',
                                  'KW',


                                  'MW',
                                  'MV',
                                  'ML',
                                  'MH',
                                  'FM',
                                  'NA',
                                  'NR',
                                  'NE',
                                  'PW',
                                  'PG',
                                  'WS',
                                  'SM',
                                  'ST',
                                  'SN',
                                  'SC',
                                  'SL',
                                  'SB',
                                  'KN',
                                  'LC',
                                  'VC',
                                  'SR',
                                  'TL',
                                  'TO',
                                  'TT',
                                  'TV',


                                            'NA',
                                            'NE',
                                            'NG',
                                            'NI',
                                            'NL',
                                            'NO',
                                            'NP',
                                            'NR',
                                            'NZ',
                                            'OM',
                                            'PA',
                                            'PE',
                                            'PG',
                                            'PH',
                                            'PK',
                                            'PL',
                                            'PS',
                                            'PT',
                                            'PW',
                                            'PY',


...

                                            'PL',
                                            'PS',
                                            'PT',
                                            'PW',
                                            'PY',
                                            'QA',
                                            'RO',
                                            'RS',
                                            'RW',
                                            'SA',
                                            'SB',
                                            'SC',
                                            'SE',
                                            'SG',
                                            'SI',
                                            'SK',
                                            'SL',
                                            'SN',
                                            'SR',
                                            'ST',


            'external_urls': {'spotify': 'https://open.spotify.com/track/2Eeur20xVqfUoM3Q7EFPFt'},
            'href': 'https://api.spotify.com/v1/tracks/2Eeur20xVqfUoM3Q7EFPFt',
            'id': '2Eeur20xVqfUoM3Q7EFPFt',
            'is_local': False,
            'name': 'the last great american dynasty',
            'popularity': 76,
            'preview_url': None,
            'track': True,
            'track_number': 3,
            'type': 'track',
            'uri': 'spotify:track:2Eeur20xVqfUoM3Q7EFPFt'},
  'video_thumbnail': {'url': None}},
 {'added_at': '2023-05-25T02:24:36Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/clgvhwn4sxd81bxao64txfapn'},
               'href': 'https://api.spotify.com/v1/users/clgvhwn4sxd81bxao64txfapn',
               'id': 'clgvhwn4sxd81bxao64txfapn',
               'type': 'user',
               'uri': 'spotify:user:clgvhwn4sxd81bxao64txfapn'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'al

                                  'SG',
                                  'SK',
                                  'ES',
                                  'SE',
                                  'CH',
                                  'TW',
                                  'TR',
                                  'UY',
                                  'US',
                                  'GB',
                                  'AD',
                                  'LI',
                                  'MC',
                                  'ID',
                                  'JP',
                                  'TH',
                                  'VN',
                                  'RO',
                                  'IL',
                                  'ZA',
                                  'SA',
                                  'AE',
                                  'BH',
                                  'QA',
                                  'OM',


                                  'JP',
                                  'TH',
                                  'VN',
                                  'RO',
                                  'IL',
                                  'ZA',
                                  'SA',
                                  'AE',
                                  'BH',
                                  'QA',
                                  'OM',
                                  'KW',
                                  'EG',
                                  'MA',
                                  'DZ',
                                  'TN',
                                  'LB',
                                  'JO',
                                  'PS',
                                  'IN',
                                  'BY',
                                  'KZ',
                                  'MD',
                                  'UA',
                                  'AL',


                      'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4qwGe91Bz9K2T8jXTZ815W'},
                                   'href': 'https://api.spotify.com/v1/artists/4qwGe91Bz9K2T8jXTZ815W',
                                   'id': '4qwGe91Bz9K2T8jXTZ815W',
                                   'name': 'Janet Jackson',
                                   'type': 'artist',
                                   'uri': 'spotify:artist:4qwGe91Bz9K2T8jXTZ815W'}],
                      'available_markets': ['AD',
                                            'AE',
                                            'AG',
                                            'AL',
                                            'AM',
                                            'AO',
                                            'AR',
                                            'AT',
                                            'AU',
                                            'AZ',
        

In [83]:
# Get Genre from each Album ID in separate JSON request
genres = []

#for album in album_ids:
#    query_url = url+'albums/'+album
#    json_req = requests.get(query_url, headers=headers)
#    response = json_req.json()
#    genres.append(response['genres'])
query_url = url+'artists/'+'2YZyLoL8N0Wb9xBt1NhZWg'
json_req = requests.get(query_url, headers=headers)
response = json_req.json()
pp.pprint(response)
#genres.append(response['album']['genres'])
#print(genres)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/2YZyLoL8N0Wb9xBt1NhZWg'},
 'followers': {'href': None, 'total': 24034094},
 'genres': ['conscious hip hop', 'hip hop', 'rap', 'west coast rap'],
 'href': 'https://api.spotify.com/v1/artists/2YZyLoL8N0Wb9xBt1NhZWg',
 'id': '2YZyLoL8N0Wb9xBt1NhZWg',
 'images': [{'height': 640,
             'url': 'https://i.scdn.co/image/ab6761610000e5eb437b9e2a82505b3d93ff1022',
             'width': 640},
            {'height': 320,
             'url': 'https://i.scdn.co/image/ab67616100005174437b9e2a82505b3d93ff1022',
             'width': 320},
            {'height': 160,
             'url': 'https://i.scdn.co/image/ab6761610000f178437b9e2a82505b3d93ff1022',
             'width': 160}],
 'name': 'Kendrick Lamar',
 'popularity': 89,
 'type': 'artist',
 'uri': 'spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg'}
